# Very quick and notebook for preparing data for training

In [84]:
import pandas as pd
from pathlib import Path
import csv

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [74]:
ddir = Path("DATA/")
train_d= Path("DATA/TRAIN")
proc_d= Path("DATA/PROC")

In [75]:
test_d = pd.read_csv(proc_d/"proc_data_test_wresult.csv")
train_d = pd.read_csv(proc_d/"proc_data_train_wresult.csv")

### Tokenize result and moves

In [76]:
t = Tokenizer(num_words=None, filters=' ', lower=False, split=' ', char_level=False)
t.fit_on_texts(list(train_d['moves']) + list(test_d['moves']))
print(len(t.word_index))

4688


In [86]:
x_t = t.texts_to_sequences(train_d['moves'])
x_v = t.texts_to_sequences(test_d['moves'])
max_len = max([len(x) for x in x_t + x_v])
x_t = pad_sequences(x_t, maxlen=max_len, padding='post', truncating='post', value=0)
x_v = pad_sequences(x_v, maxlen=max_len, padding='post', truncating='post', value=0)

In [87]:
max_len

496

In [88]:
print(x_t[0])
print(train_d['moves'][0])
print()
print()
print(x_v[0])
print(test_d['moves'][0])
print()
print()
print(t.word_index['O-O'])
print(t.word_index['Nf3'])
print(t.word_index['e4'])
print(t.word_index['1/2-1/2'])
print(t.word_index['<N>'])

[  1   4   3   1  11   9   1  40  14   1 109  31   1  57   2   1  41  44
   1   2  62   1   7  12   1  63  53   1 201   8   1  83  83   1 385 117
   1 303 353   1 689 118   1 624 149   1   5  32   1  42 449   1 134 152
   1 173 352  59   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

### Super simple model prototyping

In [109]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(4096, input_shape=(max_len,)),
    Activation('relu'),
    Dense(2048),
    Activation('relu'),
    Dense(2048),
    Activation('relu'),
    Dense(1024),
    Activation('relu'),
    Dense(256),
    Dense(1)
])

model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [110]:
model.fit(x_t, train_d['white'], epochs=50, batch_size=128, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 3s 156us/step - loss: 404525228.5660 - val_loss: 3032049.2996
Epoch 2/50
20000/20000 [==============================] - 2s 109us/step - loss: 1458960.2797 - val_loss: 414245.7762
Epoch 3/50
20000/20000 [==============================] - 2s 109us/step - loss: 531327.7062 - val_loss: 333993.7346
Epoch 4/50
20000/20000 [==============================] - 2s 113us/step - loss: 440396.4542 - val_loss: 324948.9475
Epoch 5/50
20000/20000 [==============================] - 2s 113us/step - loss: 384029.6008 - val_loss: 396347.1310
Epoch 6/50
20000/20000 [==============================] - 2s 115us/step - loss: 360686.2912 - val_loss: 485115.8086
Epoch 7/50
20000/20000 [==============================] - 2s 116us/step - loss: 334055.0983 - val_loss: 311329.0245
Epoch 8/50
20000/20000 [==============================] - 2s 113us/step - loss: 306896.9003 - val_loss: 313295.1004
Epoch 9/50
20000/2

In [111]:
train_y = train_d['white']
preds = model.predict(x_v)
preds[0:5], train_y[0:5]

(array([[1710.9651],
        [1996.2219],
        [2975.2708],
        [1776.8636],
        [2119.1755]], dtype=float32), 0    2354
 1    2523
 2    1915
 3    2446
 4    2168
 Name: white, dtype: int64)

In [80]:
train_d

,Event,result,white,black,moves,ratings
0,1,1/2-1/2,2354,2411,<N> Nf3 Nf6 <N> c4 c5 <N> b3 g6 <N> Bb2 Bg7 <N...,18 17 12 8 -5 12 3 -2 22 21 20 13 8 21 11 3 -6...
1,2,1/2-1/2,2523,2460,<N> e4 e5 <N> Nf3 Nf6 <N> d4 Nxe4 <N> Nxe5 d6 ...,26 44 26 18 14 34 36 31 37 35 42 52 55
2,3,0-1,1915,1999,<N> e4 d5 <N> exd5 Nf6 <N> d4 Nxd5 <N> Nf3 g6 ...,26 51 68 57 65 77 48 93 61 63 63 58 53 46 69 2...
3,4,1-0,2446,2191,<N> c4 Nf6 <N> Nc3 d6 <N> d4 e5 <N> Nf3 Nbd7 <...,2 21 5 53 35 45 37 54 10 22 8 48 30 17 13 35 -...
4,5,1-0,2168,2075,<N> e4 c5 <N> Nf3 d6 <N> b4 Nf6 <N> bxc5 Nxe4 ...,26 64 35 53 18 20 18 20 10 49 60 95 91 82 83 9...
5,6,1/2-1/2,2437,2254,<N> Nf3 d5 <N> d4 Nf6 <N> c4 e6 <N> Nc3 Be7 <N...,18 29 10 22 20 18 19 28 24 18 17 16 18 38 48 3...
6,7,1-0,2449,2201,<N> Nf3 Nf6 <N> g3 g6 <N> Bg2 Bg7 <N> c4 d5 <N...,21 11 -4 26 22 15 17 34 26 32 6 45 34 32 32 45...
7,8,1/2-1/2,1813,1643,<N> e4 c5 <N> Nf3 d6 <N> d4 cxd4 <N> Qxd4 Nc6 ...,26 64 35 53 39 38 41 22 15 14 18 10 18 -4 5 21...
8,9,1-0,2553,2052,<N> e4 e6 <N> d4 d5 <N> Nc3 Nf6 <N> Bg5 Bb4 <N...,26 42 43 29 31 28 20 15 5 -3 -2 0 5 51 48 46 4...
9,10,1-0,2611,2520,<N> e4 e6 <N> d4 d5 <N> Nd2 c5 <N> Ngf3 a6 <N>...,26 42 43 29 -5 22 22 36 35 21 14 36 35 18 31 4...


In [102]:
test_d

,Event,result,moves,ratings
0,25001,1/2-1/2,<N> e4 e5 <N> Nc3 Nc6 <N> Nf3 Nf6 <N> g3 Bc5 <...,26 44 23 16 20 23 -34 -18 -6 -13 -12 -1 3 2 -2...
1,25002,0-1,<N> d4 c5 <N> d5 Nf6 <N> c4 e6 <N> Nc3 exd5 <N...,19 52 48 61 51 48 54 63 50 45 59 89 70 62 38 3...
2,25003,1-0,<N> d4 Nf6 <N> Nc3 g6 <N> e4 d6 <N> Be3 Bg7 <N...,19 22 13 71 79 63 48 47 29 33 27 62 68 77 33 6...
3,25004,1/2-1/2,<N> d4 Nf6 <N> c4 c5 <N> d5 e6 <N> Nc3 exd5 <N...,19 22 16 50 51 48 54 63 58 45 59 95 74 59 60 6...
4,25005,1/2-1/2,<N> e4 e6 <N> d4 d5 <N> Nc3 Nc6 <N> Bb5 Nf6 <N...,26 48 46 29 25 47 32 37 -10 -3 -8 -8 -18 -6 -1...
5,25006,0-1,<N> d4 d6 <N> c4 e5 <N> Nc3 Nc6 <N> d5 Nce7 <N...,19 48 38 27 2 65 58 78 66 67 49 60 49 63 39 35...
6,25007,1-0,<N> e4 c5 <N> Nf3 a6 <N> c3 d5 <N> exd5 Qxd5 <...,26 64 35 65 48 51 62 52 65 51 44 45 49 71 53 9...
7,25008,1/2-1/2,<N> Nf3 b6 <N> e4 Bb7 <N> Nc3 e6 <N> d4 Bb4 <N...,15 77 87 56 57 63 52 74 81 63 43 61 44 61 41 3...
8,25009,1-0,<N> e4 e5 <N> Nf3 Nc6 <N> Bb5 a6 <N> Ba4 Nf6 <...,13 44 20 14 30 31 -1 11 -18 -7 -17 12 21 55 18...
9,25010,1-0,<N> d4 Nf6 <N> Nf3 b5 <N> Bg5 Bb7 <N> Nbd2 c5 ...,19 22 8 94 77 91 74 92 77 92 94 141 88 95 80 1...
